# GAN Image Generation

In [12]:
# Importing libraries
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

In [13]:
# Discriminator Model
def define_discriminator(
    in_shape=(28,28,1)
):
    model = Sequential()
    model.add(
        Conv2D(
            64, 
            (3,3), 
            strides=(2, 2), 
            padding='same', 
            input_shape=in_shape
        )
    )
    model.add(
        LeakyReLU(
            alpha=0.2
        )
    )
    model.add(
        Dropout(
            0.4
        )
    )
    model.add(
        Conv2D(
            64, 
            (3,3), 
            strides=(2, 2), 
            padding='same'
        )
    )
    model.add(
        LeakyReLU(
            alpha=0.2
        )
    )
    model.add(
        Dropout(
            0.4
        )
    )
    model.add(
        Flatten()
    )
    model.add(
        Dense(
            1, 
            activation='sigmoid'
        )
    )
    opt = Adam(
        lr=0.0002, 
        beta_1=0.5
    )
    model.compile(
        loss='binary_crossentropy', 
        optimizer=opt, 
        metrics=['accuracy']
    )
    return model

In [14]:
# Generator Model
def define_generator(
    latent_dim
):
    model = Sequential()
    n_nodes = 128 * 7 * 7
    model.add(
        Dense(
            n_nodes, 
            input_dim=latent_dim
        )
    )
    model.add(
        LeakyReLU(
            alpha=0.2
        )
    )
    model.add(
        Reshape(
            (7, 7, 128)
        )
    )
    model.add(
        Conv2DTranspose(
            128, 
            (4,4), 
            strides=(2,2), 
            padding='same'
        )
    )
    model.add(
        LeakyReLU(
            alpha=0.2
        )
    )
    model.add(
        Conv2DTranspose(
            128, 
            (4,4), 
            strides=(2,2), 
            padding='same'
        )
    )
    model.add(
        LeakyReLU(
            alpha=0.2
        )
    )
    model.add(
        Conv2D(
            1, 
            (7,7), 
            activation='sigmoid', 
            padding='same'
        )
    )
    return model

In [15]:
# Combining Generator and Discriminator Models
def define_gan(
    g_model, 
    d_model
):
    d_model.trainable = False
    model = Sequential()
    model.add(
        g_model
    )
    model.add(
        d_model
    )
    opt = Adam(
        lr=0.0002, 
        beta_1=0.5
    )
    model.compile(
        loss='binary_crossentropy', 
        optimizer=opt
    )
    return model

In [16]:
# Load Training Data
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = expand_dims(
        trainX, 
        axis=-1
    )
    X = X.astype(
        'float32'
    )
    X = X / 255.0
    return X

In [17]:
# Generate Real Samples
def generate_real_samples(
    dataset, 
    n_samples
):
    ix = randint(
        0, 
        dataset.shape[0], 
        n_samples
    )
    X = dataset[ix]
    y = ones(
        (
            n_samples, 
            1
        )
    )
    return X, y

In [18]:
# Generate Latent Space Points
def generate_latent_points(
    latent_dim, 
    n_samples
):
    x_input = randn(
        latent_dim * n_samples
    )
    x_input = x_input.reshape(
        n_samples, 
        latent_dim
    )
    return x_input

In [19]:
# Generate Fake Examples With Class Labels
def generate_fake_samples(
    g_model, 
    latent_dim, 
    n_samples
):
    x_input = generate_latent_points(
        latent_dim, 
        n_samples
    )
    X = g_model.predict(
        x_input
    )
    y = zeros(
        (n_samples, 1)
    )
    return X, y

In [20]:
# Save Plot of Generated Images
def save_plot(
    examples, 
    epoch, 
    n=10
):
    for i in range(n * n):
        pyplot.subplot(
            n, 
            n, 
            1 + i
        )
        pyplot.axis(
            'off'
        )
        pyplot.imshow(
            examples[i, :, :, 0], 
            cmap='gray_r'
        )
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(
        filename
    )
    pyplot.close()

In [21]:
# Evaluate The Discriminator, Then Plot the Generated Images, then Save the Generator Model
def summarize_performance(
    epoch, 
    g_model, 
    d_model, 
    dataset, 
    latent_dim, 
    n_samples=100
):
    X_real, y_real = generate_real_samples(
        dataset, 
        n_samples
    )
    _, acc_real = d_model.evaluate(
        X_real, 
        y_real, 
        verbose=0
    )
    x_fake, y_fake = generate_fake_samples(
        g_model, 
        latent_dim, 
        n_samples
    )
    _, acc_fake = d_model.evaluate(
        x_fake, 
        y_fake, 
        verbose=0
    )
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    save_plot(
        x_fake, 
        epoch
    )
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(
        filename
    )

In [24]:
# Train the Generator and Discriminator
def train(
    g_model, 
    d_model, 
    gan_model, 
    dataset, 
    latent_dim, 
    n_epochs=10, 
    n_batch=256
):
    bat_per_epo = int(
        dataset.shape[0] / n_batch
    )
    half_batch = int(
        n_batch / 2
    )
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(
                dataset, 
                half_batch
            )
            X_fake, y_fake = generate_fake_samples(
                g_model, 
                latent_dim, 
                half_batch
            )
            X = vstack(
                (X_real, X_fake)
            )
            y = vstack(
                (y_real, y_fake)
            )
            d_loss, _ = d_model.train_on_batch(
                X, 
                y
            )
            X_gan = generate_latent_points(
                latent_dim, 
                n_batch
            )
            y_gan = ones(
                (n_batch, 1)
            )
            g_loss = gan_model.train_on_batch(
                X_gan, 
                y_gan
            )
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        if (i+1) % 10 == 0:
            summarize_performance(
                i, 
                g_model, 
                d_model, 
                dataset, 
                latent_dim
            )

In [ ]:
# Running the Training
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(
    latent_dim
)
gan_model = define_gan(
    g_model, 
    d_model
)
dataset = load_real_samples()
train(
    g_model, 
    d_model, 
    gan_model, 
    dataset, 
    latent_dim
)